<a href="https://colab.research.google.com/github/mitchelllierman/GovRuleQuery/blob/main/GovRuleQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Planning

Blueprint:

Problem:
Need a lot of applications for government exemptions/permits?
Don't know how to find the records you're looking for?



## Components


### Goal

Understanding of the records you want to obtain

### Prompt



*   Description of records
*   Instruction to generate regulations.gov query
*   regulations.gov query documentation for additional context
*   Example record for additional context
*   Potential example of paired document with successful query (for higher-shot)


### Model

Claude 3.7 for now
Potentially finetuned Llama or other model instead?
Allow people to connect their API key for their own model if wanted?

### Assessment & Storage

Opt-in mechanism that appends the resulting prompt/query pair to a csv so that future people looking for same records can have a framework to start from
Allow for scoring of prompt/query pairs somehow? Allow people to see what isn't work well and what is.

# Code

## Setup

Before you are able to use this tool, you will need a regulations.gov API key. Those keys are available to apply for [on the API documentation page here](https://open.gsa.gov/api/regulationsgov/)!

In [ ]:
import requests
import pandas as pd


API_KEY = input("Enter your API key: ")


if len(API_KEY) < 40:
  print("Invalid API Key, defaulting to the Demo Key provided by regulations.gov.\n
  Please provide the correct key to avoid low rate limits.")
  API_KEY = "DEMO_KEY"
else:
  print("Valid API Key")

BASE_URL = "https://api.regulations.gov/v4/documents"

### Test

In [6]:
# Query parameters for FAA Part 137 applications
params = {
    "filter[searchTerm]": "(FAA Part 137) AND (UAS) AND (hereby grant)",
    "page[size]": 10,  # Adjust as needed
    "api_key": API_KEY,
    "sort": "-postedDate",
}

# Make the API request
response = requests.get(BASE_URL, params=params)

if response.status_code == 200:
    data = response.json()
    # Flatten the JSON response into a DataFrame
    df = pd.json_normalize(data['data'])

    # Save to CSV
    csv_path = "/content/FAA_Part_137_Applications.csv"
    df.to_csv(csv_path, index=False)

    print(f"CSV saved at: {csv_path}")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

CSV saved at: /content/FAA_Part_137_Applications.csv


In [7]:
test = pd.read_csv("/content/FAA_Part_137_Applications.csv")
test.head()

,id,type,attributes.documentType,attributes.lastModifiedDate,attributes.highlightedContent,attributes.frDocNum,attributes.withdrawn,attributes.agencyId,attributes.allowLateComments,attributes.commentEndDate,attributes.title,attributes.postedDate,attributes.docketId,attributes.subtype,attributes.commentStartDate,attributes.withinCommentPeriod,attributes.openForComment,attributes.objectId,links.self
0,FAA-2022-0178-0009,documents,Other,2024-11-20T18:49:39Z,NaN,NaN,False,FAA,True,NaN,U.S. DOT/FAA - Decision,2024-11-20T05:00:00Z,FAA-2022-0178,Decision,2024-11-20T05:00:00Z,False,True,090000648682ab3f,https://api.regulations.gov/v4/documents/FAA-2...
1,FAA-2024-2613-0002,documents,Other,2024-11-20T14:25:34Z,NaN,NaN,False,FAA,True,NaN,Ceres Air - Exemption/Rulemaking (Additional I...,2024-11-20T05:00:00Z,FAA-2024-2613,Additional Information,2024-11-20T05:00:00Z,False,True,090000648682a73d,https://api.regulations.gov/v4/documents/FAA-2...
2,FAA-2022-0700-0006,documents,Other,2024-11-20T15:54:56Z,NaN,NaN,False,FAA,True,NaN,U.S. DOT/FAA - Decision,2024-11-20T05:00:00Z,FAA-2022-0700,Decision,2024-11-20T05:00:00Z,False,True,0900006486829058,https://api.regulations.gov/v4/documents/FAA-2...
3,FAA-2020-0470-0006,documents,Other,2024-11-20T21:02:42Z,NaN,NaN,False,FAA,True,NaN,"L&M Road Services, LLC - Exemption/Rulemaking ...",2024-11-20T05:00:00Z,FAA-2020-0470,Amendment,2024-11-20T05:00:00Z,False,True,090000648682b1c6,https://api.regulations.gov/v4/documents/FAA-2...
4,FAA-2022-0319-0017,documents,Other,2025-03-01T23:52:25Z,NaN,NaN,False,FAA,True,NaN,U.S. DOT/FAA - Decision,2024-11-20T05:00:00Z,FAA-2022-0319,Decision,2024-11-20T05:00:00Z,False,True,0900006486828d50,https://api.regulations.gov/v4/documents/FAA-2...


This test proves that you can use the groupings and boolean operators available on the regulations.gov website via the searchTerm query parameter in the request URL we are building using the requests library. Results should be consistent across either option.

### Generalizing

TODO